# Neighbourhood Finder demo

# Imports and helper functions

In [9]:
import json
from random import randint
import os
import json
import lyricwikia
import pickle
import sys
import numpy as np

def artists_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['artist_name'])
    return list(set(all_))

def albums_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['album_name'])
    return list(set(all_))

def tracks_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_uri'])
    return list(set(all_))

def track_names_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_name'])
    return list(set(all_))

def lyrics_in_playlist(playlist):
    all_ = {}
    info = {}
    ct = 0
    failed = []
    for track in playlist['tracks']:
        track_uri = track['track_uri']
        song_name = track['track_name']
        artist_name = track['artist_name']
        try:
            lyrics = lyricwikia.get_lyrics(artist_name, song_name)
            all_[track_uri] = lyrics
            info[track_uri] = (song_name,artist_name)
        except:
            failed.append(ct)
        ct += 1
    return all_,failed,info

f = open('../models/uri_to_track_info.pckl','rb')
uri_to_name_artist = pickle.load(f)
f.close()

f = open('../models/artist_to_songs.p','rb')
artist_to_songs = pickle.load(f)
f.close()

clean_artist_to_songs = {}
for artist in artist_to_songs.keys():
    clean_artist_to_songs[artist] = list(set(artist_to_songs[artist]))

name_artist_to_uri = {}
for uri in uri_to_name_artist.keys():
    tup = uri_to_name_artist[uri]
    name_artist_to_uri[tup] = uri

from gensim.models import Word2Vec
model = Word2Vec.load('../models/model_min5_new.bin')

# Pick song name by looking up for artist

In [10]:
clean_artist_to_songs['Coldplay']

['Everglow - Single Version',
 'Hypnotised - EP Mix',
 'Up in Flames - Live',
 'Mylo Xyloto - Live',
 'Shiver - Live In Sydney',
 'Us Against the World - Live',
 'Lovers In Japan - Osaka Sun Mix',
 "Everything's Not Lost - Live In Sydney",
 'Careful Where You Stand',
 'Everglow',
 'U.F.O.',
 'Ink - Live At Le Casino De Paris, Paris',
 'Life In Technicolor ii - Live @ The O2, London',
 'Every Teardrop Is a Waterfall - Live',
 'Midnight',
 'Death And All His Friends',
 'Crests Of Waves',
 'Every Teardrop Is a Waterfall - Live from Glastonbury 2011',
 'A Sky Full Of Stars - Radio Edit',
 'Everglow - Single Version, Radio Edit',
 'ブラザーズ・アンド・シスターズ',
 'Christmas Lights',
 'A Message',
 'Mylo Xyloto',
 'Yellow - Live From Spotify London',
 'Army Of One',
 'See You Soon - Live In Sydney',
 'Viva La Vida - Live',
 'A Sky Full of Stars',
 'Twisted Logic',
 'A Head Full of Dreams',
 'A Spell A Rebel Yell',
 'God Put A Smile Upon Your Face',
 'Princess of China - Live',
 "Another's Arms - Live At 

In [8]:
list_already = []
list_s = [('Warriors','Imagine Dragons'),('The Scientist','Coldplay'),('Immigrant Song','Led Zeppelin'),('T.N.T.','AC/DC')]
song_names = []
artist_names = []
for l in list_s:
    song_names.append(l[0])
    artist_names.append(l[1])
    uri = name_artist_to_uri[l].split(':')[-1]
    list_already.append(uri)
sample = np.zeros(300)
ct = 0
for song in list_already:
    
    track = 'spotify:track:%s'%song
    try:
        ct += 1
        sample += model[track]
    except:
        print(uri_to_name_artist[track],track)
representation = sample/ct

most_sim = model.most_similar([representation],topn=1000)
artists = []
songs = []
for suggestion in most_sim:
    
    songs.append(uri_to_name_artist[suggestion[0]])
    artists.append(uri_to_name_artist[suggestion[0]][1])
#     print(uri_to_name_artist[suggestion[0]])

from collections import Counter
c = Counter(artists)
top_artists = c.most_common(10)

# print('Top songs - ')
# print(*songs[:25],sep='\n')
# print('\n\n\nTop Artisis')
# for t in top_artists:
#     print(t[0])

most_sim = model.most_similar([representation],topn=1000)
artists = []
songs = []
for suggestion in most_sim:
    
    songs.append(uri_to_name_artist[suggestion[0]])
    artists.append(uri_to_name_artist[suggestion[0]][1])
#     print(uri_to_name_artist[suggestion[0]])

from collections import Counter
c = Counter(artists)
top_artists = c.most_common(14)

print('Top songs - ')
printed = 0
i = 0
while(printed<25):
    song,artist = songs[i]
    if song not in song_names:
        if artist not in artist_names:
            print(song,artist)
            printed += 1
    i += 1
            
print('\n\n\nTop Artisis')
for t in top_artists:
    if t not in artist_names:
        print(t[0])

Top songs - 
Iron Man - Remastered Version Black Sabbath
Crazy Train - Remastered Ozzy Osbourne
In The End Black Veil Brides
Smoke On the Water Deep Purple
Hail to the King Avenged Sevenfold
Seven Nation Army The White Stripes
Through The Fire And Flames DragonForce
Seven Nation Army Wiesnrocker
I'd Love To Change The World - Matstubs Remix Jetta
U.S.A. - The Stars Spangled Banner, National Anthem, Nationalhymne, Hymne National, Himno Nacional, национальный гимн National Anthem
Sweet Victory Baseball Hockey Sports Crew
Detroit Rock City KISS
Immortals Fall Out Boy
Rock You Like A Hurricane Scorpions
Paranoid - Remastered Version Black Sabbath
The Final Countdown Europe
Centuries Fall Out Boy
My Demons Starset
Hell Yeah Rev Theory
I Am Machine Three Days Grace
I Don't Care Fall Out Boy
Shell Shocked (feat. Kill The Noise & Madsonik) - From "Teenage Mutant Ninja Turtles" Juicy J
Painkiller Three Days Grace
Afterlife Avenged Sevenfold
Twin Skeleton's (Hotel In NYC) Fall Out Boy



Top Art

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
